<a href="https://colab.research.google.com/github/importYM/proposal/blob/main/NWPPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Google Colab environment**

In [ ]:
!nvidia-smi -L

In [ ]:
!nvidia-smi

# **1. Pandas import & data load**
* pandas import
* csv file read
* data print

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pandas import read_csv

df = pd.read_csv('/content/sample_221212_4.csv') # read csv file 
df.columns, df, df.shape[1]

(Index(['TIME', 'G3', 'ID', 'F1', 'F2', 'F3'], dtype='object'),
     TIME  G3  ID      F1      F2     F3
 0      1   3   1  151.00   63.00  20.00
 1      2   3   1  151.00   63.00  20.00
 2      3   2   1  121.00   51.00  20.00
 3      4   2   1  155.00   57.00  20.00
 4      5   2   1  132.00   56.00  20.00
 5      1   1   2   93.00   61.00  18.00
 6      2   2   2   93.00   61.00  18.00
 7      3   2   2   93.00   61.00  18.00
 8      4   2   2  101.00   62.00  18.00
 9      5   2   2  101.00   62.00  18.00
 10     1   3   3  133.00   60.00  18.00
 11     2   3   3  120.00   68.00  18.00
 12     3   3   3  113.00   63.00  18.00
 13     4   3   3  117.00   66.00  18.00
 14     5   3   3  139.67   71.33  18.00
 15     1   3   4  119.00   84.00  15.00
 16     2   3   4  119.00  103.00  20.00
 17     3   3   4  128.75   80.00  18.75
 18     4   2   4  128.75   80.00  18.75
 19     5   2   4  128.75   80.00  18.75
 20     1   4   5  156.20   49.00  19.20
 21     2   4   5  135.67   47.71 

# **3. Normalization feature scaling**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

df_for_scaler = df.iloc[:, 3:]                                 # select column for normalization(TIME, nursing workload proxies grade = ID열 제외하고 4열부터 마지막열까지 선택)

scaler = MinMaxScaler(feature_range=(0, 1))                    # scaler setting
scaled_data = scaler.fit_transform(df_for_scaler)              # scaler fit ->MinMaxScaler는 numpy array를 반환한다

scaled_df = pd.DataFrame(scaled_data, columns=df.columns[3:])  # 반환된 numpy array를 다시 DataFrame으로 변환하면서 columns name을 이전 데이터와 동일하게 설정
non_scaled_df = df.iloc[:, 1:3]                                # select column (nursing workload proxies grade, ID = df의 두번째와 세번째열 추출)
scaled_df_all = pd.concat([non_scaled_df, scaled_df], axis=1)  # non_scaled_df & scaled_df merge

print(scaled_df_all)

    G3  ID        F1        F2     F3
0    3   1  0.917722  0.276542  1.000
1    3   1  0.917722  0.276542  1.000
2    2   1  0.443038  0.059504  1.000
3    2   1  0.981013  0.168023  1.000
4    2   1  0.617089  0.149937  1.000
5    1   2  0.000000  0.240369  0.600
6    2   2  0.000000  0.240369  0.600
7    2   2  0.000000  0.240369  0.600
8    2   2  0.126582  0.258455  0.600
9    2   2  0.126582  0.258455  0.600
10   3   3  0.632911  0.222283  0.600
11   3   3  0.427215  0.366974  0.600
12   3   3  0.316456  0.276542  0.600
13   3   3  0.379747  0.330801  0.600
14   3   3  0.738449  0.427202  0.600
15   3   4  0.411392  0.656357  0.000
16   3   4  0.411392  1.000000  1.000
17   3   4  0.565665  0.584012  0.750
18   2   4  0.565665  0.584012  0.750
19   2   4  0.565665  0.584012  0.750
20   4   5  1.000000  0.023332  0.840
21   4   5  0.675158  0.000000  0.572
22   2   5  0.553797  0.258455  1.000
23   2   5  0.189873  0.222283  0.800
24   2   5  0.324367  0.439320  0.600
25   2   6  

# **4. Setting the Maximum Data Length (hospital day 30)**

In [ ]:
'''

final_df = pd.DataFrame(columns=scaled_df_all.columns)         # 빈 데이터프레임 생성

for id in scaled_df_all['ID'].unique():                        # 각 인덱스별로 루프를 돌면서 ID마다 행 수를 맞추기 위한 for문
    rows = scaled_df_all[scaled_df_all['ID'] == id]
    n_rows = len(rows)
    n_padding_rows = (31 - n_rows % 31) % 31                   # 31의 배수가 되도록 패딩할 행의 수 계산 31= Maximum Data Length(hospital day 30)+1(ID구분 위한 선)
    padding_rows = pd.DataFrame({'ID': [id] * n_padding_rows,
                                 'G3': [0] * n_padding_rows,
                                 'F1': [0] * n_padding_rows,
                                 'F2': [0] * n_padding_rows,
                                 'F3': [0] * n_padding_rows})  # 패딩할 행의 값은 0으로 설정
    new_rows = pd.concat([rows, padding_rows])
    new_rows = new_rows.iloc[:31]                              # 인덱스별로 최대 31개의 행만 남기고 나머지는 버림
    final_df = pd.concat([final_df, new_rows])

# 결과 확인
print(final_df)

'''

In [ ]:
final_df = pd.DataFrame(columns=scaled_df_all.columns)         # 빈 데이터프레임 생성

for id in scaled_df_all['ID'].unique():                        # 각 인덱스별로 루프를 돌면서 ID마다 행 수를 맞추기 위한 for문
    rows = scaled_df_all[scaled_df_all['ID'] == id]
    n_rows = len(rows)
    n_padding_rows = (8 - n_rows % 8) % 8                      # 8의 배수가 되도록 패딩할 행의 수 계산 8= Maximum Data Length(hospital day 7)+1(ID구분 위한 선)
    padding_rows = pd.DataFrame({'ID': [id] * n_padding_rows,
                                 'G3': [0] * n_padding_rows,
                                 'F1': [0] * n_padding_rows,
                                 'F2': [0] * n_padding_rows,
                                 'F3': [0] * n_padding_rows})  # 패딩할 행의 값은 0으로 설정
    new_rows = pd.concat([rows, padding_rows])
    new_rows = new_rows.iloc[:8]                               # 인덱스별로 최대 8개의 행만 남기고 나머지는 버림
    final_df = pd.concat([final_df, new_rows])

# 결과 확인
print(final_df)

   G3 ID        F1        F2     F3
0   3  1  0.917722  0.276542  1.000
1   3  1  0.917722  0.276542  1.000
2   2  1  0.443038  0.059504  1.000
3   2  1  0.981013  0.168023  1.000
4   2  1  0.617089  0.149937  1.000
0   0  1  0.000000  0.000000  0.000
1   0  1  0.000000  0.000000  0.000
2   0  1  0.000000  0.000000  0.000
5   1  2  0.000000  0.240369  0.600
6   2  2  0.000000  0.240369  0.600
7   2  2  0.000000  0.240369  0.600
8   2  2  0.126582  0.258455  0.600
9   2  2  0.126582  0.258455  0.600
0   0  2  0.000000  0.000000  0.000
1   0  2  0.000000  0.000000  0.000
2   0  2  0.000000  0.000000  0.000
10  3  3  0.632911  0.222283  0.600
11  3  3  0.427215  0.366974  0.600
12  3  3  0.316456  0.276542  0.600
13  3  3  0.379747  0.330801  0.600
14  3  3  0.738449  0.427202  0.600
0   0  3  0.000000  0.000000  0.000
1   0  3  0.000000  0.000000  0.000
2   0  3  0.000000  0.000000  0.000
15  3  4  0.411392  0.656357  0.000
16  3  4  0.411392  1.000000  1.000
17  3  4  0.565665  0.584012

In [ ]:
final_df=final_df.reset_index(drop=True)   # 중복 인덱스를 삭제하기 위해 drop=True를 사용하여 이전 인덱스를 삭제하고 새 인덱스를 생성함
print(final_df)

   G3 ID        F1        F2     F3
0   3  1  0.917722  0.276542  1.000
1   3  1  0.917722  0.276542  1.000
2   2  1  0.443038  0.059504  1.000
3   2  1  0.981013  0.168023  1.000
4   2  1  0.617089  0.149937  1.000
5   0  1  0.000000  0.000000  0.000
6   0  1  0.000000  0.000000  0.000
7   0  1  0.000000  0.000000  0.000
8   1  2  0.000000  0.240369  0.600
9   2  2  0.000000  0.240369  0.600
10  2  2  0.000000  0.240369  0.600
11  2  2  0.126582  0.258455  0.600
12  2  2  0.126582  0.258455  0.600
13  0  2  0.000000  0.000000  0.000
14  0  2  0.000000  0.000000  0.000
15  0  2  0.000000  0.000000  0.000
16  3  3  0.632911  0.222283  0.600
17  3  3  0.427215  0.366974  0.600
18  3  3  0.316456  0.276542  0.600
19  3  3  0.379747  0.330801  0.600
20  3  3  0.738449  0.427202  0.600
21  0  3  0.000000  0.000000  0.000
22  0  3  0.000000  0.000000  0.000
23  0  3  0.000000  0.000000  0.000
24  3  4  0.411392  0.656357  0.000
25  3  4  0.411392  1.000000  1.000
26  3  4  0.565665  0.584012

# **5. Split data into Train / Test**

In [ ]:
from sklearn.model_selection import train_test_split            # package import

x = final_df.iloc[:, 0:]                                        # 0열부터 맨 끝 열까지 추출하여 feature set(x)로 설정 (nursing workload proxies grade, ID, Biomarker, Hospital demographic data
y = final_df.iloc[:, 0:2]                                       # 0~1열 추출하여 target set(y)로 설정 (nursing workload proxies grade, ID)

train_ids, test_ids = train_test_split(final_df['ID'].unique(), # train/test set으로 분할하기위해 기준이 되는 고유 ID 값을 추출하되,
                                       train_size=0.8,          # train/test 분할 비율은 80%로,
                                       random_state=0)          # random분할 하지 않도록 설정

train_idx = final_df['ID'].isin(train_ids)                      # final_df 데이터셋에서 train용 ID로 인덱스 추출하여 train set을 설정
test_idx = final_df['ID'].isin(test_ids)                        # final_df 데이터셋에서 test용 ID로 인덱스 추출하여 test set을 설정

x_train, x_test = x[train_idx], x[test_idx]                     # x feature set에서 train_idx, test_idx에 해당하는 인덱스를 추출하여 x_train, x_test로 분리
y_train, y_test = y[train_idx], y[test_idx]                     # y feature set에서 train_idx, test_idx에 해당하는 인덱스를 추출하여 y_train, y_test로 분리

print(x_train, x_test, y_train, y_test)

   G3 ID        F1        F2     F3
0   3  1  0.917722  0.276542  1.000
1   3  1  0.917722  0.276542  1.000
2   2  1  0.443038  0.059504  1.000
3   2  1  0.981013  0.168023  1.000
4   2  1  0.617089  0.149937  1.000
5   0  1  0.000000  0.000000  0.000
6   0  1  0.000000  0.000000  0.000
7   0  1  0.000000  0.000000  0.000
8   1  2  0.000000  0.240369  0.600
9   2  2  0.000000  0.240369  0.600
10  2  2  0.000000  0.240369  0.600
11  2  2  0.126582  0.258455  0.600
12  2  2  0.126582  0.258455  0.600
13  0  2  0.000000  0.000000  0.000
14  0  2  0.000000  0.000000  0.000
15  0  2  0.000000  0.000000  0.000
24  3  4  0.411392  0.656357  0.000
25  3  4  0.411392  1.000000  1.000
26  3  4  0.565665  0.584012  0.750
27  2  4  0.565665  0.584012  0.750
28  2  4  0.565665  0.584012  0.750
29  0  4  0.000000  0.000000  0.000
30  0  4  0.000000  0.000000  0.000
31  0  4  0.000000  0.000000  0.000
32  4  5  1.000000  0.023332  0.840
33  4  5  0.675158  0.000000  0.572
34  2  5  0.553797  0.258455

# **6. Change Pandas DataFrame to Numpy array**

In [ ]:
x_train = x_train.values   # Change Pandas DataFrame to Numpy array
x_test = x_test.values     # Change Pandas DataFrame to Numpy array
y_train = y_train.values   # Change Pandas DataFrame to Numpy array
y_test = y_test.values     # Change Pandas DataFrame to Numpy array

print(x_train, x_test, y_train, y_test)

[[3 1 0.917721518987342 0.2765418701392657 1.0]
 [3 1 0.917721518987342 0.2765418701392657 1.0]
 [2 1 0.44303797468354444 0.059504431181045425 1.0]
 [2 1 0.9810126582278482 0.1680231506601555 1.0]
 [2 1 0.6170886075949367 0.14993669741363713 1.0]
 [0 1 0.0 0.0 0.0]
 [0 1 0.0 0.0 0.0]
 [0 1 0.0 0.0 0.0]
 [1 2 0.0 0.24036896364622895 0.6000000000000001]
 [2 2 0.0 0.24036896364622895 0.6000000000000001]
 [2 2 0.0 0.24036896364622895 0.6000000000000001]
 [2 2 0.12658227848101267 0.2584554168927473 0.6000000000000001]
 [2 2 0.12658227848101267 0.2584554168927473 0.6000000000000001]
 [0 2 0.0 0.0 0.0]
 [0 2 0.0 0.0 0.0]
 [0 2 0.0 0.0 0.0]
 [3 4 0.4113924050632911 0.6563573883161511 0.0]
 [3 4 0.4113924050632911 1.0 1.0]
 [3 4 0.5656645569620256 0.5840115753300779 0.75]
 [2 4 0.5656645569620256 0.5840115753300779 0.75]
 [2 4 0.5656645569620256 0.5840115753300779 0.75]
 [0 4 0.0 0.0 0.0]
 [0 4 0.0 0.0 0.0]
 [0 4 0.0 0.0 0.0]
 [4 5 1.0000000000000002 0.023331524688008698 0.8399999999999999]
 [4